<a href="https://colab.research.google.com/github/Inefra21/Colour-bias-experiment/blob/main/Test_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing models

# Importing packages

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Loading data and base model

In [ ]:
# Mount the drive to access files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the trained model
# Change file path
model = tf.keras.models.load_model(
  '/content/drive/MyDrive/model_90.keras',
  custom_objects={'KerasLayer': hub.KerasLayer})

In [ ]:
# Create directory to test images
test_lcat_dir = '/content/drive/MyDrive/test_cats/light'
test_dcat_dir = '/content/drive/MyDrive/test_cats/dark'
test_ldog_dir = '/content/drive/MyDrive/test_dogs/light'
test_ddog_dir = '/content/drive/MyDrive/test_dogs/dark'
test_mcat_dir = '/content/drive/MyDrive/test_cats/mixed'
test_mdog_dir = '/content/drive/MyDrive/test_dogs/mixed'

## Understanding the data

In [ ]:
# Count the images in each folder
# You must add the a string to the end of the directory because of the structure of the dataset
num_lcat = len(os.listdir(test_lcat_dir + '/light'))
num_dcat = len(os.listdir(test_dcat_dir + '/dark'))
num_ldog = len(os.listdir(test_ldog_dir + '/light'))
num_ddog = len(os.listdir(test_ddog_dir + '/dark'))

num_mcat = len(os.listdir(test_mcat_dir + '/mixed'))
num_mdog = len(os.listdir(test_mdog_dir + '/mixed'))

total_cats = num_lcat + num_dcat + num_mcat
total_dogs = num_ldog + num_ddog + num_mdog

In [ ]:
print('total test cat images:', total_cats)
print('total test dog images:', total_dogs)

# Setting model parameters

In [ ]:
# The image generator will produce batches of 50 images of shape 150 pixels by 150 pixels
IMG_SHAPE = 150
BATCH_SIZE1 = 50
BATCH_SIZE2 = 500

# Creating batched datasets

In [ ]:
image_gen_test = ImageDataGenerator(rescale=1./255)

# Image generator with light cats
test_lcat_gen = image_gen_test.flow_from_directory(batch_size=BATCH_SIZE1,
                                                 directory=test_lcat_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')

# Image generator with dark cats
test_dcat_gen = image_gen_test.flow_from_directory(batch_size=BATCH_SIZE1,
                                                 directory=test_dcat_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')

# Image generator with light dogs
test_ldog_gen = image_gen_test.flow_from_directory(batch_size=BATCH_SIZE1,
                                                 directory=test_ldog_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')

# Image generator with dark dogs
test_ddog_gen = image_gen_test.flow_from_directory(batch_size=BATCH_SIZE1,
                                                 directory=test_ddog_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')

In [ ]:
# Batch size is set to 500 as there are more images
# Image generator with mixed cat images
test_mcat_gen = image_gen_test.flow_from_directory(batch_size=BATCH_SIZE2,
                                                 directory=test_mcat_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')

# Image generator with mixed dog images
test_mdog_gen = image_gen_test.flow_from_directory(batch_size=BATCH_SIZE2,
                                                 directory=test_mdog_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')

# Testing the models

In [ ]:
# Store results of predictions
lcat_predicted = tf.squeeze(model.predict(test_lcat_gen)).numpy()
dcat_predicted = tf.squeeze(model.predict(test_dcat_gen)).numpy()
ldog_predicted = tf.squeeze(model.predict(test_ldog_gen)).numpy()
ddog_predicted = tf.squeeze(model.predict(test_ddog_gen)).numpy()

# Calculate the label with the highest probability for each prediction
lcat_predicted_lb = np.argmax(lcat_predicted, axis=-1)
dcat_predicted_lb = np.argmax(dcat_predicted, axis=-1)
ldog_predicted_lb = np.argmax(ldog_predicted, axis=-1)
ddog_predicted_lb = np.argmax(ddog_predicted, axis=-1)

In [ ]:
# Create variables to count the number of mistakes
lcat_mistakes = 0
dcat_mistakes = 0
ldog_mistakes = 0
ddog_mistakes = 0

# Count number of mistakes for each category
for prediction in range(num_lcat):
  if lcat_predicted_lb[prediction] == 1:
    lcat_mistakes += 1

for prediction in range(num_dcat):
  if dcat_predicted_lb[prediction] == 1:
    dcat_mistakes += 1

for prediction in range(num_ldog):
  if ldog_predicted_lb[prediction] == 0:
    ldog_mistakes += 1

for prediction in range(num_ddog):
  if ddog_predicted_lb[prediction] == 0:
    ddog_mistakes += 1

# Print results
print("Accuracy light cats:", 100 - (lcat_mistakes*100/num_lcat), "%")
print("Accuracy dark cats:", 100 - (dcat_mistakes*100/num_dcat), "%")
print("Accuracy light dogs:", 100 - (ldog_mistakes*100/num_ldog), "%")
print("Accuracy dark dogs:", 100 - (ddog_mistakes*100/num_ddog), "%")

In [ ]:
# Predict labels for mixed cat images
# Save the images for visualising later
mcat_images, mcat_labels = next(iter(test_mcat_gen))
mcat_predicted = tf.squeeze(model.predict(mcat_images)).numpy()
mcat_predicted_lb = np.argmax(mcat_predicted, axis=-1)

# Predict labels for mixed dog images
# Save the images for visualising later
mdog_images, mdog_labels = next(iter(test_mdog_gen))
mdog_predicted = tf.squeeze(model.predict(mdog_images)).numpy()
mdog_predicted_lb = np.argmax(mdog_predicted, axis=-1)

In [ ]:
# Create array to store the misclassified images
mcat_mistakes = []
mdog_mistakes = []

for prediction in range(num_mcat):
  if mcat_predicted_lb[prediction] == 1:
    mcat_mistakes.append(mcat_images[prediction])

for prediction in range(num_mdog):
  if mdog_predicted_lb[prediction] == 0:
    mdog_mistakes.append(mdog_images[prediction])

# Print results
print("Accuracy on cats:", 100 - (len(mcat_mistakes)*100/num_mcat), "%")
print("Accuracy on dogs:", 100 - (len(mdog_mistakes)*100/num_mdog), "%")

# Visualizing some of the misclassified images

In [ ]:
counter = 1

# Show the cat images which were classified as dogs
plt.figure(figsize=(10, 9))
for n, img in enumerate(mcat_mistakes):
  if n < 30:
    plt.subplot(6, 5, counter)
    plt.subplots_adjust(hspace = 0.3)
    plt.imshow(mcat_images[n])
    plt.axis('off')
    counter += 1
_ = plt.suptitle("Misclassified cat images")

In [ ]:
counter = 1

# Show the dog images which were classified as cats
plt.figure(figsize=(10, 9))
for n, img in enumerate(mdog_mistakes):
  if n < 30:
  plt.subplot(6, 5, counter)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(mdog_images[n])
  plt.axis('off')
  counter += 1
_ = plt.suptitle("Misclassified dog images")